In [1]:
using Flux, Flux.Data.MNIST
using Flux: onehotbatch, argmax, crossentropy, throttle
using Base.Iterators: repeated, partition

LoadError: [91mArgumentError: Module CLArrays not found in current path.
Run `Pkg.add("CLArrays")` to install the CLArrays package.[39m

In [2]:
X_train = MNIST.images()
y_train = onehotbatch(MNIST.labels(), 0:9)
X_test = MNIST.images(:test)
y_test = onehotbatch(MNIST.labels(:test), 0:9)

10×10000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false  false  false   true  false  …  false  false  false  false  false
 false  false   true  false  false     false  false  false  false  false
 false   true  false  false  false      true  false  false  false  false
 false  false  false  false  false     false   true  false  false  false
 false  false  false  false   true     false  false   true  false  false
 false  false  false  false  false  …  false  false  false   true  false
 false  false  false  false  false     false  false  false  false   true
  true  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false

In [15]:
X_test_f32 = [CLArray(Float32.(X_test[i])) for i in 1:10_000]

10000-element Array{CLArrays.CLArray{Float32,2},1}:
 GPU: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 GPU: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 GPU: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 GPU: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 GPU: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 GPU: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 GPU: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 GPU: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 GPU: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 GPU: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 GPU: Fl

In [ ]:
test_data = [(cat(4, (X_test_f32[i])...), y_test[:,i])
         for i in partition(1:10_000, 100)]

In [ ]:
# Partition into batches of size 1,000
training_data = [(cat(4, float.(X_train[i])...), y_train[:,i])
         for i in partition(1:60_000, 100)]



In [12]:
size(X_train)
size(X_test)

# X_train = cat(4, float.(X_train)...)
X_test = cat(4, float.(X_test)...)

28×28×1×10000 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0       …  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0       …  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.329412     0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.870588     0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.262745     0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0       …  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 ⋮  

In [13]:
println(size(X_test))
println(size(X_train[1]))
println(size(training_data[1][1]))
println(typeof(training_data[1][1]))
println(size(y_test))
println(typeof(y_test))

(28, 28, 1, 10000)
(28, 28)


LoadError: [91mUndefVarError: training_data not defined[39m

In [4]:
model = Chain(
            #28 * 28
          Conv2D((3, 3), 1=>32, relu),
            #26 * 26 * 32   
          x -> maxpool2d(x, 2),
            #13 * 13 * 32   
          Conv2D((3, 3), 32=>64, relu),
            #11 * 11 * 64
          x -> maxpool2d(x, 2),
            #5 * 5 * 64
          x -> reshape(x, :, size(x, 4)),
          Dense(1600, 128, relu),
          Dense(128, 10), 
          softmax)

Chain(Conv2D((3, 3), 1=>32, NNlib.relu), #3, Conv2D((3, 3), 32=>64, NNlib.relu), #4, #5, Dense(1600, 128, NNlib.relu), Dense(128, 10), NNlib.softmax)

In [5]:
# Use cross entropy loss here
loss(x, y) = crossentropy(model(x), y)
accuracy(x, y) = mean(argmax(model(x)) .== argmax(y))

accuracy (generic function with 1 method)

In [43]:
model(training_data[1][1])

Tracked 10×100 Array{Float64,2}:
 0.100653   0.10109    0.10045    …  0.1008     0.100539   0.100422 
 0.0999416  0.0999198  0.100076      0.099995   0.0999518  0.100784 
 0.0994649  0.0995074  0.0994752     0.0993689  0.0995463  0.0994048
 0.0999704  0.0995734  0.0998853     0.0996269  0.100258   0.0997994
 0.0999763  0.0999218  0.100443      0.099772   0.0997275  0.0999368
 0.0993883  0.0994419  0.0993712  …  0.0991524  0.0991812  0.0992823
 0.100322   0.10038    0.100296      0.100341   0.100149   0.100043 
 0.0998979  0.0998368  0.100124      0.0999648  0.1        0.0999722
 0.0998985  0.0999473  0.0996546     0.100265   0.0999678  0.0999566
 0.100486   0.100383   0.100226      0.100715   0.10068    0.1004   

In [7]:
# Callback during the training
function eval_cb()
    
#     println("Training loss: $(loss(X_train, y_train)), acc: $(accuracy(X_train, y_train))")
    println("Test loss: $(loss(X_test, y_test)), acc: $(accuracy(X_test, y_test))")
    println()
end

optimizer = ADAM(params(model))

Flux.train!(loss, training_data, optimizer, cb = throttle(eval_cb, 10))

LoadError: [91mInterruptException:[39m

In [ ]:
println("Optimization Finished!")
@printf "Model accuracy on test set: %.3f\n" accuracy(X_test, y_test)